# Impact of age of enrollment on the Five Domains of Speech-Language in Children with Hearing Loss at age 4 years

Paper 2

In [1]:
# Import modules and set options
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import arviz as az

sns.set(context='notebook', style='ticks')

Import data

In [2]:
lsl_dr = (pd.read_csv('../data/clean/lsl_dr.csv', index_col=0, low_memory=False)
                  .rename({'onset_1':'identify_mo'}, axis=1))

In [3]:
lsl_dr.head()

,redcap_event_name,academic_year_rv,hl,male,_race,prim_lang,sib,_mother_ed,father_ed,par1_ed,...,gf3_sis_ss,gf3_siw_ss,gf_version,ppvt_f,ppvt_ss,rowpvt_ss,school,score,test_name,test_type
0,initial_assessment_arm_1,2002.0,0.0,0.0,0.0,0.0,1.0,6.0,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,101,58.0,NaN,EOWPVT
1,initial_assessment_arm_1,2002.0,0.0,0.0,0.0,0.0,1.0,6.0,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,101,51.0,PLS,receptive
2,initial_assessment_arm_1,2002.0,0.0,0.0,0.0,0.0,1.0,6.0,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,101,60.0,PLS,expressive
3,initial_assessment_arm_1,2002.0,0.0,0.0,0.0,0.0,1.0,6.0,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,101,54.0,PLS,total
4,initial_assessment_arm_1,2011.0,0.0,1.0,3.0,0.0,1.0,5.0,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,626,96.0,NaN,EOWPVT


Indicator for non-profound hearing loss

In [4]:
lsl_dr['deg_hl_below6'] = lsl_dr.degree_hl<6
lsl_dr.loc[lsl_dr.degree_hl.isnull(), 'deg_hl_below6'] = np.nan

Indicator for first intervention outside OPTION

In [5]:
lsl_dr['int_outside_option'] = lsl_dr.age > lsl_dr.age_int
lsl_dr.loc[lsl_dr.age < lsl_dr.age_int, 'int_outside_option'] = np.nan

Indicator for high school graduation of mother

In [6]:
lsl_dr['mother_college'] = lsl_dr.mother_ed >= 3
lsl_dr.loc[lsl_dr.mother_ed.isnull(), 'mother_college'] = None

Create age in years variable

In [7]:
lsl_dr['age_years'] = lsl_dr.age/12.

Create school index

In [8]:
schools_unique = np.sort(lsl_dr.school.unique())
school_lookup = dict(zip(schools_unique, range(len(schools_unique))))

In [9]:
lsl_dr['school_idx'] = lsl_dr.school.replace(school_lookup)

Create student index

In [10]:
student_unique = np.sort(lsl_dr.study_id.unique())
student_lookup = dict(zip(student_unique, range(len(student_unique))))

In [11]:
lsl_dr['student_idx'] = lsl_dr.study_id.replace(student_lookup)

Add expressive and receptive to langauge test domains

In [12]:
lsl_dr.loc[(lsl_dr.domain=='Language') & (lsl_dr.test_type=='receptive'), 
           'domain'] = 'Receptive Language'
lsl_dr.loc[(lsl_dr.domain=='Language') & (lsl_dr.test_type=='expressive'), 
           'domain'] = 'Expressive Language'

In [13]:
lsl_dr['ident_3mo'] = lsl_dr.identify_mo <=3
lsl_dr.loc[lsl_dr.identify_mo.isnull(), 'ident_3mo'] = None
lsl_dr['int_6mo'] = lsl_dr.age_int <= 6
lsl_dr.loc[lsl_dr.identify_mo.isnull(), 'int_6mo'] = None

Drop records with missing race and age at ernollment, since there is less than 1% of them

In [14]:
lsl_dr = lsl_dr.dropna(subset=['race', 'age_years', 'int_outside_option'])

### Exclusions

Drop non-english and other disabilities, filter for hearing loss

In [15]:
other_etiology = (lsl_dr[['etiology_3___2', 'etiology_3___4', 'etiology_3___5', 'etiology_3___6', 'etiology_3___9',
       'etiology_oth___1', 'etiology_oth___3', 'etiology_oth___4', 'etiology_oth___8', 'etiology_oth___9']]
                      .sum(1).astype(bool))
NON_ENGLISH = lsl_dr.non_english.astype(bool) 

In [16]:
SECONDARY_DISABILITY = (lsl_dr.synd_or_disab.astype(bool) 
     | ~(lsl_dr.etiology_2.isin([0,4]))
     | (lsl_dr.etiology_2.isnull() & other_etiology))

In [17]:
# no secondary distability, english first language
inclusion_mask_1 = ((lsl_dr.degree_hl!=0)
                  & ~SECONDARY_DISABILITY
                  & ~NON_ENGLISH)

# secondary disability, english first language
inclusion_mask_2 = ((lsl_dr.degree_hl!=0)
                  & SECONDARY_DISABILITY
                  & ~NON_ENGLISH)

# no secondary disability, non-english
inclusion_mask_3 = ((lsl_dr.degree_hl!=0)
                  & ~SECONDARY_DISABILITY
                  & NON_ENGLISH)

# secondary disability, non-english
inclusion_mask_4 = ((lsl_dr.degree_hl!=0)
                  & SECONDARY_DISABILITY
                  & NON_ENGLISH)

In [18]:
masks = {'no secondary disability, english': inclusion_mask_1, 
         'secondary disability, english': inclusion_mask_2, 
         'no secondary disability, non-english': inclusion_mask_3, 
         'secondary disability, non-english': inclusion_mask_4}

In [19]:
covariates = ['score', 'student_idx', 'school_idx', 'male', 'sib', 'family_inv', 'race', 'age_test', 
              'domain', 'deg_hl_below6', 'mother_college', 'age_years', 'test_type', 'time', 'bilateral_ci',
              'bilateral_ha', 'unilateral_ci', 'unilateral_ha', 'bimodal', 'age_amp',
              'ident_3mo', 'int_outside_option', 'int_6mo']

In [23]:
group = 'no secondary disability, english'
filename_stub = ''

In [51]:
analysis_subset = lsl_dr.loc[masks[group], covariates].copy().dropna(subset=['time', 'score', 'age_test']) 
analysis_subset.shape[0]

31272

In [52]:
unique_students = analysis_subset.drop_duplicates(subset='student_idx')
unique_students.shape[0]

3060

In [53]:
null_prop = analysis_subset.isnull().mean()
null_prop[null_prop>0].round(2).sort_values(ascending=False)

mother_college    0.25
age_amp           0.15
int_6mo           0.12
ident_3mo         0.12
family_inv        0.09
deg_hl_below6     0.07
sib               0.05
dtype: float64

## Model specification

In [54]:
def fillna(x, value):
    x_masked = np.ma.masked_invalid(x)
    np.ma.set_fill_value(x_masked, value)
    return x_masked

fillna(np.array([0.4, np.nan, 5]), 0.5)

masked_array(data=[0.4, --, 5.0],
             mask=[False,  True, False],
       fill_value=0.5)

In [55]:
from pymc3 import Bernoulli, Normal, Uniform, Dirichlet, Categorical, Beta, HalfCauchy
from pymc3 import Gamma, Exponential, Multinomial, HalfNormal, NormalMixture, Lognormal
from pymc3 import Model, Deterministic, Metropolis
from numpy.ma import masked_values, set_fill_value, masked_invalid
import theano.tensor as tt
from theano import shared

def generate_model(dataset, cohort_age=4):
    
    if cohort_age==2:
        mask = (dataset.age_test>=24) & (dataset.age_test<36)
    elif cohort_age==3:
        mask = (dataset.age_test>=36) & (dataset.age_test<48)
    elif cohort_age==4:
        mask = (dataset.age_test>=48) & (dataset.age_test<60)
    elif cohort_age==5:
        mask = (dataset.age_test>=60) & (dataset.age_test<72)
    elif cohort_age==6:
        mask = (dataset.age_test>=72) & (dataset.age_test<84)
    else:
        print('Invalid age!')
        return
    
    # Generate mean scores
    mean_scores = dataset[mask].groupby('student_idx').score.mean()
    dataset_unique = dataset[mask].drop_duplicates(subset='student_idx')
    dataset_unique.set_index('student_idx').drop('score', axis=1).join(mean_scores)
    assert not dataset_unique.score.isnull().sum()
    
    (family_inv, school, time, sib,
             mother_college, age_amp,
             ident_3mo,
             int_outside_option, int_6mo, score) = dataset_unique[['family_inv', 
                                                     'school_idx', 
                                                    'time', 'sib', 'mother_college', 
                                                    'age_amp', 
                                                    'ident_3mo', 'int_outside_option', 'int_6mo',
                                                                   'score']].astype(float).T.values

    int_option = (~int_outside_option.astype(bool)).astype(int)

    with Model() as model:
        
        # Imputation of age of amplification
        if np.isnan(age_amp).sum():
            m_age_amp = Normal("m_age_amp", 0, sd=5, shape=2)
            s_age_amp = Exponential("s_age_amp", 1)
            p_age_amp = Beta('p_age_amp', 1, 1)
            _x_age_amp = NormalMixture('x_age_amp', [p_age_amp, 1-p_age_amp], m_age_amp, 
                                       sd=s_age_amp,
                                       observed=masked_invalid(np.log(age_amp+0.1)))
            x_age_amp = (tt.exp(_x_age_amp) - 0.1) / 12
        else:
            x_age_amp = age_amp / 12
        
        # Imputation of family involvement
        if np.isnan(family_inv).sum():
            p_family_inv = Dirichlet("p_family_inv", np.ones(5))
            x_family_inv = Categorical('x_family_inv', p_family_inv, 
                                       observed=masked_invalid(family_inv))
        else:
            x_family_inv = family_inv
            
        # Imputation of siblings
        if np.isnan(sib).sum():
            n_sib_cats = len(dataset.sib.unique())
            p_sib = Dirichlet("p_sib", np.ones(n_sib_cats))
            x_sib = Categorical('x_sib', p_sib, observed=masked_invalid(sib))
        else:
            x_sib = sib
            
        # Imputation of 3 month identification
        if np.isnan(ident_3mo).sum():
            p_3mo = Beta("p_3mo", 1, 1)
            x_3mo = Bernoulli('x_3mo', p_3mo, observed=masked_invalid(ident_3mo))
        else:
            x_3mo = ident_3mo
            
        # Imputation of 6 month intervention
        if np.isnan(int_6mo).sum():
            p_6mo = Beta("p_6mo", 1, 1)
            x_6mo = Bernoulli('x_6mo', p_6mo, observed=masked_invalid(int_6mo))
        else:
            x_6mo = int_6mo
            
        # Indices to school random effects
        unique_schools = np.unique(school)
        school_index = [list(unique_schools).index(s) for s in school]

        # School random effect (non-centered parameterization)
        μ_school = Normal('μ_school', 90, sd=10)
        σ_school = Exponential("σ_school", 1)
        z_school = Normal('z_school', mu=0, sd=1, shape=len(unique_schools))
        α_school = Deterministic("α_school", μ_school + z_school*σ_school)
        
        # Random intercepts
        intercept = α_school[school_index]

        # Covariates
        X = [x_age_amp, 
             x_family_inv, 
             x_sib,
             mother_college, 
             time,
             x_3mo,
             x_6mo,
             int_option,
             int_option*x_6mo]
        

        # Fixed effects
        β = Normal("β", 0, sd=100, shape=len(X))
        θ = intercept + β.dot(tt.stack(X))
        σ = HalfNormal("σ", sd=25, testval=100)
        score_like = Normal("score_like", mu=θ, sd=σ, observed=score)

    return model

## Receptive Language Test Score Model

In [56]:
receptive_language_dataset = analysis_subset[(analysis_subset.domain=='Receptive Language')]

receptive_language_dataset.shape[0]

6013

In [57]:
receptive_language_4 = generate_model(receptive_language_dataset, 4)

/home/fonnesbeck_gmail_com/anaconda3/envs/dev/lib/python3.7/site-packages/pymc3/model.py:1331: UserWarning: Data in x_age_amp contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, UserWarning)
/home/fonnesbeck_gmail_com/anaconda3/envs/dev/lib/python3.7/site-packages/pymc3/model.py:1331: UserWarning: Data in x_family_inv contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, UserWarning)
/home/fonnesbeck_gmail_com/anaconda3/envs/dev/lib/python3.7/site-packages/pymc3/model.py:1331: UserWarning: Data in x_sib contains missing values and will be automatically imputed from the sampling distribution.
  warnings.warn(impute_message, UserWarning)
/home/fonnesbeck_gmail_com/anaconda3/envs/dev/lib/python3.7/site-packages/pymc3/model.py:1331: UserWarning: Data in x_3mo contains missing values and will be automatically imputed from the sampling distribution.
  warn

In [58]:
iterations = 1000
tuning = 4000

In [ ]:
mother_college    0.25
age_amp           0.15
int_6mo           0.12
ident_3mo         0.12
family_inv        0.09
deg_hl_below6     0.07
sib               0.05

In [59]:
CHECK_MODEL = True

if CHECK_MODEL:
    print(receptive_language_4.check_test_point())

m_age_amp                         -5.06
s_age_amp_log__                   -1.06
p_age_amp_logodds__               -1.39
x_age_amp_missing                  0.00
p_family_inv_stickbreaking__      -4.87
x_family_inv_missing               0.00
p_sib_stickbreaking__             -4.87
x_sib_missing                      0.00
p_3mo_logodds__                   -1.39
x_3mo_missing                      0.00
p_6mo_logodds__                   -1.39
x_6mo_missing                      0.00
μ_school                          -3.22
σ_school_log__                    -1.06
z_school                         -37.68
β                                -49.72
σ_log__                           -6.84
x_age_amp                      -6419.58
x_family_inv                   -1633.58
x_sib                          -1633.58
x_3mo                           -703.54
x_6mo                           -703.54
score_like                          NaN
Name: Log-probability of test_point, dtype: float64


In [49]:
from pymc3 import sample

with receptive_language_4:
    rec_lang_4_trace = sample(iterations, tune=tuning, chains=2, cores=2)

Multiprocess sampling (2 chains in 2 jobs)
CompoundStep
>NUTS: [σ, β, z_school, σ_school, μ_school, p_6mo, p_3mo, p_sib, p_family_inv, x_age_amp_missing, p_age_amp, s_age_amp, m_age_amp]
>CategoricalGibbsMetropolis: [x_sib_missing, x_family_inv_missing]
>BinaryGibbsMetropolis: [x_3mo_missing, x_6mo_missing]
Sampling 2 chains:   0%|          | 0/10000 [00:00<?, ?draws/s]INFO (theano.gof.compilelock): Waiting for existing lock by process '4149' (I am process '4148')
INFO (theano.gof.compilelock): To manually release the lock, delete /home/fonnesbeck_gmail_com/.theano/compiledir_Linux-4.9--amd64-x86_64-with-debian-9.6--3.7.3-64/lock_dir
/home/fonnesbeck_gmail_com/anaconda3/envs/dev/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)

Bad initial energy, check any log probabilities that are inf or -inf, nan or very small:
score_like   NaN


ParallelSamplingError: Bad initial energy

/home/fonnesbeck_gmail_com/anaconda3/envs/dev/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


In [ ]:
labels = ['Age at amplification',
          'Family Involvement Score', 
          'Number of siblings',
          'Mother with College Ed',
          'Years in program',
          'Identified <= 3mo',          
          'Intervention <= 6mo',
          'Intervention with OPTION',
          'Interaction']

In [ ]:
_,axes = az.plot_forest(rec_lang_4_trace, 
               var_names=['β'],
              combined=True)
axes[0].set_title('Receptive Language')
axes[0].vlines(0, *axes[0].get_ylim(), linestyles='dotted')
axes[0].set_yticklabels(labels[::-1]);

In [ ]:
from pymc3 import forestplot

forestplot(rec_lang_4_trace, varnames=['β'], ylabels=labels, main='Receptive Language')

In [ ]:
forestplot(rec_lang_4_trace, varnames=['α_school'])

In [ ]:
from pymc3 import effective_n

effective_n(rec_lang_4_trace)

In [ ]:
from pymc3 import energyplot

energyplot(rec_lang_4_trace)

In [ ]:
forestplot(rec_lang_4_trace, varnames=['β'], ylabels=labels, main='Receptive Language')

In [ ]:
forestplot(rec_lang_4_trace, varnames=['β_race'], ylabels=['Black', 'Hispanic', 'Asian', 'Other'])

The school random effect standard deviation is a measure of how variable scores are among schools. The estimated standard deviation is about 4 points for this domain.

In [ ]:
from pymc3 import traceplot

traceplot(rec_lang_4_trace, varnames=['σ_school'])

In [ ]:
from pymc3 import summary

summary(rec_lang_4_trace, varnames=['β']).set_index(pd.Index(labels))

In [ ]:
forestplot(rec_lang_4_trace, varnames=["predictions"], rhat=False, 
           ylabels=['Student {}'.format(i) for i in range(1,5)],
           main='Predicted receptive language scores');

## Expressive Language Model

In [ ]:
expressive_language_dataset = analysis_subset[(analysis_subset.domain=='Expressive Language')]

In [ ]:
expressive_language_4 = generate_model(expressive_language_dataset, 4)

In [ ]:
with expressive_language_4:
    
    exp_lang_4_trace = sample(iterations, tune=tuning)

In [ ]:
forestplot(exp_lang_4_trace, varnames=['β'], ylabels=labels, main='Receptive Language')

In [ ]:
forestplot(exp_lang_4_trace, varnames=["predictions"], rhat=False, 
           ylabels=['Student {}'.format(i) for i in range(1,5)],
           main='Predicted receptive language scores')

## Articulation Model

In [ ]:
articulation_dataset = analysis_subset[(analysis_subset.domain=='Articulation')]

In [ ]:
articulation_4 = generate_model(articulation_dataset, 4)

In [ ]:
with articulation_4:
    
    artic_4_trace = sample(iterations, tune=tuning)

In [ ]:
forestplot(artic_4_trace, varnames=['β'], ylabels=labels, main='Receptive Language')

In [ ]:
forestplot(artic_4_trace, varnames=["predictions"], rhat=False, 
           ylabels=['Student {}'.format(i) for i in range(1,5)],
           main='Predicted receptive language scores')

## Expressive Vocabulary Model

In [ ]:
expressive_vocab_dataset = analysis_subset[(analysis_subset.domain=='Expressive Vocabulary')]

In [ ]:
expressive_vocab_4 = generate_model(expressive_vocab_dataset, 4)

In [ ]:
with expressive_vocab_4:
    
    expressive_vocab_4_trace = sample(iterations, tune=tuning)

In [ ]:
forestplot(expressive_vocab_4_trace, varnames=['β'], ylabels=labels, main='Receptive Language')

In [ ]:
forestplot(expressive_vocab_4_trace, varnames=["predictions"], rhat=False, 
           ylabels=['Student {}'.format(i) for i in range(1,5)],
           main='Predicted receptive language scores')

## Receptive Vocabulary Model

In [ ]:
receptive_vocab_dataset = analysis_subset[(analysis_subset.domain=='Receptive Vocabulary')]

In [ ]:
receptive_vocab_4 = generate_model(receptive_vocab_dataset, 4)

In [ ]:
with receptive_vocab_4:
    
    receptive_vocab_4_trace = sample(iterations, tune=tuning)

In [ ]:
forestplot(receptive_vocab_4_trace, varnames=['β'], ylabels=labels, main='Receptive Language')

In [ ]:
forestplot(receptive_vocab_4_trace, varnames=["predictions"], rhat=False, 
           ylabels=['Student {}'.format(i) for i in range(1,5)],
           main='Predicted receptive language scores')